In [ ]:
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt


N = 10000
np.random.seed(42)

x0 = np.random.exponential(scale=1.0, size=N)
x1 = np.random.lognormal(mean=0.0, sigma=1.0, size=N)
x2 = np.random.normal(loc=0.0, scale=1.0, size=N)
x3 = np.random.normal(loc=0.0, scale=1.0, size=N)
x4 = np.random.normal(loc=0.0, scale=1.0, size=N)

# Add strong outliers to 1% of x2
outlier_idx = np.random.choice(N, size=int(0.01 * N), replace=False)
x2[outlier_idx] *= 10.0  # Amplify outliers

# Uniform distribution for stable features
x5 = np.random.uniform(-1, 1, size=N)
x6 = np.random.uniform(-1, 1, size=N)
x7 = np.random.uniform(-1, 1, size=N)
x8 = np.random.uniform(-1, 1, size=N)
x9 = np.random.uniform(-1, 1, size=N)

# 2. Define target function (nonlinear + noise)
def custom_f(x0, x1, x2, x3, x4):
    return (
        2.0 * np.log1p(x0)    # log(1 + x0) to handle long-tail
        + 0.5 * np.sqrt(x1)   # sqrt(x1) to smooth lognormal
        - 3.0 * np.cos(x2)    # periodic effect
        + 2.0 * x3**2         # quadratic term
        + np.tanh(x4)         # nonlinearity
    )

noise = np.random.normal(loc=0, scale=1.0, size=N)
y = custom_f(x0, x1, x2, x3, x4) + noise

# 3. Convert to DataFrame
df = pd.DataFrame({
    'x0': x0, 'x1': x1, 'x2': x2, 'x3': x3, 'x4': x4,
    'x5': x5, 'x6': x6, 'x7': x7, 'x8': x8, 'x9': x9,
    'y': y
})

X = df.drop(columns=['y'])
y = df['y']


X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


model = xgb.XGBRegressor(
    objective='reg:squarederror',
    n_estimators=100,
    learning_rate=0.1,
    max_depth=4,
    random_state=42
)
model.fit(X_train, y_train)


y_pred = model.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print("RMSE:", rmse)


RMSE: 1.124316239289707


In [ ]:
from itertools import combinations
from math import comb
from sklearn.linear_model import LinearRegression
import numpy as np

test_idx = 0

def kernel_shap_no_correction_single_mean_fill(model, X_train, X_test_point):
    """
    Compute Kernel SHAP values using a mean-filled baseline without correction.
    
    - The reference baseline is the mean vector of X_train.
    - SHAP values are estimated via weighted linear regression.
    """
    p = X_train.shape[1]
    mean_vector = np.mean(X_train, axis=0)  # Compute the mean of each feature
    baseline = model.predict(mean_vector.reshape(1, -1))[0]  # Predict baseline output

    subset_list = []
    for size_s in range(1, p):  # Generate all non-empty subsets
        for combi in combinations(range(p), size_s):
            subset_list.append(combi)
    num_subsets = len(subset_list)

    # Initialize matrices for regression
    bigX = np.zeros((num_subsets, p))
    bigy = np.zeros(num_subsets)
    sample_weight = np.zeros(num_subsets)

    row_idx = 0
    for subset in subset_list:
        s = len(subset)
        w_s = (p - 1) / (comb(p, s) * s * (p - s))  # Weight for subset

        mask_vec = np.zeros(p)
        mask_vec[list(subset)] = 1.0  # Mark selected features

        # Construct perturbed input by copying mean vector
        perturbed = mean_vector.copy()
        perturbed[list(subset)] = X_test_point[list(subset)]

        fx = model.predict(perturbed.reshape(1, -1))[0]  # Get model prediction
        
        bigy[row_idx] = fx - baseline  # Compute difference from baseline
        bigX[row_idx, :] = mask_vec  # Store mask vector
        sample_weight[row_idx] = w_s  # Store weight

        row_idx += 1

    # Fit weighted linear regression
    lr = LinearRegression(fit_intercept=False)
    lr.fit(bigX, bigy, sample_weight=sample_weight)
    shap_values = lr.coef_
    return baseline, shap_values


x_test_1_arr = X_test.iloc[test_idx].values

# Add small perturbations to x_test_1_arr multiple times
mean_fill_shap_values_list = []
n_runs = 100
for seed in range(n_runs):
    np.random.seed(seed)
    # Add small Gaussian noise to the input
    x_test_perturbed = x_test_1_arr + np.random.normal(0, 0.01, size=len(x_test_1_arr))
    
    baseline, shap_vals = kernel_shap_no_correction_single_mean_fill(
        model, 
        X_train.values, 
        x_test_perturbed
    )
    mean_fill_shap_values_list.append(shap_vals)

# Compute standard deviation of SHAP values across runs
shap_array = np.array(mean_fill_shap_values_list)  # shape=(n_runs, p)
stds = shap_array.std(axis=0)
print("Mean-fill SHAP feature-wise std:", stds)


Mean-fill SHAP feature-wise std: [0.00492508 0.0019016  0.09378474 0.1031966  0.0080946  0.00190162
 0.00190162 0.00190162 0.00190162 0.00190162]


In [ ]:
import numpy as np
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import pairwise_distances
from scipy.stats import truncnorm

def tabular_lime_explanation_regression(
    X_train: np.ndarray,
    model,                 
    X_test_point: np.ndarray,  
    num_samples: int = 5000,
    random_state: int = 42
):
    """
    LIME-based explanation for tabular regression models:
      1) Compute quartiles (bins) for each feature
      2) Calculate mean/std for each bin
      3) Sample from bins using truncated Gaussian distribution
      4) Compute distance-based weights from X_test_point
      5) Fit Ridge regression for local interpretation
    """
    rng = np.random.RandomState(random_state)
    n_train, n_features = X_train.shape
    
    # 0) StandardScaler for distance calculations
    scaler = StandardScaler().fit(X_train)

    # 1) Compute (min, q1, q2, q3, max) for each feature
    bin_edges_list = []
    for j in range(n_features):
        col_j = X_train[:, j]
        min_j = np.min(col_j)
        q1_j  = np.percentile(col_j, 25)
        q2_j  = np.percentile(col_j, 50)
        q3_j  = np.percentile(col_j, 75)
        max_j = np.max(col_j)
        bin_edges_list.append([min_j, q1_j, q2_j, q3_j, max_j])
    bin_edges = np.array(bin_edges_list)  # shape=(n_features, 5)

    # 2) Compute mean, std, and count for each bin
    means = np.zeros((n_features, 4))
    stds  = np.zeros((n_features, 4))
    counts = np.zeros((n_features, 4), dtype=int)

    for j in range(n_features):
        edges_j = bin_edges[j]  # [min, q1, q2, q3, max]
        col_j = X_train[:, j]

        for i_bin in range(4):
            left  = edges_j[i_bin]
            right = edges_j[i_bin+1]
            if i_bin == 0:
                mask = (col_j >= left) & (col_j <= right)
            else:
                mask = (col_j > left) & (col_j <= right)

            bin_data = col_j[mask]
            counts[j, i_bin] = len(bin_data)
            if len(bin_data) == 0:
                # If bin is empty, assign fallback values
                means[j, i_bin] = 0.5 * (left + right)
                stds[j, i_bin]  = max(1e-6, (right - left) / 6)
            else:
                means[j, i_bin] = np.mean(bin_data)
                tmp_std = np.std(bin_data, ddof=1)
                stds[j, i_bin]  = tmp_std if tmp_std >= 1e-9 else 1e-9

    # Compute bin selection probability (based on sample count)
    probs = counts / float(n_train)  # shape=(n_features, 4)

    # 3) Generate perturbed samples
    X_samples = np.zeros((num_samples, n_features))
    for i_smp in range(num_samples):
        for j in range(n_features):
            p_j = probs[j]
            bin_idx = rng.choice(4, p=p_j)  # Select bin

            m = means[j, bin_idx]
            s = stds[j, bin_idx]
            left  = bin_edges[j, bin_idx]
            right = bin_edges[j, bin_idx+1]

            if counts[j, bin_idx] == 0:
                # If bin is empty, sample from uniform distribution
                val = rng.uniform(left, right)
            else:
                # Sample from truncated normal distribution
                a = (left - m) / s
                b = (right - m) / s
                val = truncnorm.rvs(a, b, loc=m, scale=s, random_state=rng)

            X_samples[i_smp, j] = val

    # 4) Compute distance-based weights
    #    - Compute L2 distance in original scale
    dists = pairwise_distances(X_samples, X_test_point.reshape(1, -1),
                               metric='euclidean').ravel()
    kernel_width = 0.75 * np.sqrt(n_features)
    weights = np.exp(-(dists**2) / (kernel_width**2))

    # 5) Predict f(perturbed X)
    y_smp = model.predict(X_samples)  # shape: (num_samples,)

    # Fit Ridge regression for local explanation
    ridge = Ridge(alpha=1e-6, fit_intercept=True, random_state=rng)
    ridge.fit(X_samples, y_smp, sample_weight=weights)
    coefs = ridge.coef_       # shape=(n_features,)
    intercept = ridge.intercept_

    # Compute local prediction g(x_test)
    g_x_test = intercept + coefs.dot(X_test_point)

    return coefs, g_x_test


In [ ]:
# =========================================
#  (Experiment) Add small noise to x_test and compare LIME explanations
# =========================================

# Assume we have a trained XGBoost model with X_train and X_test
# test_idx = 0
# x_test_1_arr = X_test.iloc[test_idx].values

n_runs = 100
lime_coefs_list = []

for seed in range(n_runs):
    np.random.seed(seed)
    # Experiment: Add small noise to the test point
    x_test_perturbed = x_test_1_arr + np.random.normal(0, 0.01, size=len(x_test_1_arr))
    
    # Fix the internal sampling random_state (=42) to ensure consistent sampling inside LIME
    coefs, g_x_pert = tabular_lime_explanation_regression(
        X_train.values,   # shape=(N, p)
        model, 
        x_test_perturbed, 
        num_samples=5000,  # Adjust as needed
        random_state=42
    )
    lime_coefs_list.append(coefs)

# Compute feature-wise standard deviation
lime_coefs_array = np.array(lime_coefs_list)  # shape=(n_runs, p)
lime_stds = lime_coefs_array.std(axis=0)

print("== LIME (x_test with added noise) ==")
print("Feature-wise std:", lime_stds)


== LIME (x_test에 노이즈 추가) ==
Feature-wise std: [0.00098194 0.0004485  0.00592275 0.01023281 0.00057645 0.00082646
 0.00064816 0.00061897 0.00065979 0.00081536]


In [ ]:
N_samples_for_test = 100        # Number of test samples (random selection)
n_runs = 10                     # Number of iterations per sample
p = X_train.shape[1]            # Number of features

# (Results) Arrays to accumulate the sum of standard deviations for each feature
shap_std_sum_each_feature = np.zeros(p)  
lime_std_sum_each_feature = np.zeros(p)  

# Randomly select 100 test samples (without replacement)
np.random.seed(42)  # Fix random seed for reproducibility
test_indices = np.random.choice(len(X_test), size=N_samples_for_test, replace=False)

for idx in test_indices:
    # Current test sample
    x_test_original = X_test.iloc[idx].values  # shape=(p,)

    # Lists to store SHAP and LIME results across runs
    shap_values_runs = []
    lime_coefs_runs = []

    for run_id in range(n_runs):
        # Add small noise to x_test_point
        x_test_perturbed = x_test_original + np.random.normal(0, 0.01, size=p)

        # Compute SHAP values
        _, shap_vals = kernel_shap_no_correction_single_mean_fill(
            model, 
            X_train.values, 
            x_test_perturbed
        )
        shap_values_runs.append(shap_vals)

        # Compute LIME coefficients (random_state fixed)
        coefs, _ = tabular_lime_explanation_regression(
            X_train.values, 
            model, 
            x_test_perturbed,
            num_samples=3000,
            random_state=42  
        )
        lime_coefs_runs.append(coefs)

    # Convert to NumPy arrays
    shap_values_runs = np.array(shap_values_runs)  # shape=(n_runs, p)
    lime_coefs_runs = np.array(lime_coefs_runs)    # shape=(n_runs, p)

    # Compute feature-wise standard deviation
    shap_std = shap_values_runs.std(axis=0)  # shape=(p,)
    lime_std = lime_coefs_runs.std(axis=0)   # shape=(p,)

    # Accumulate feature-wise standard deviations over 100 samples
    shap_std_sum_each_feature += shap_std
    lime_std_sum_each_feature += lime_std

# At the end, shap_std_sum_each_feature and lime_std_sum_each_feature (shape=(p,))
# contain the total sum of std values recorded for each feature across 100 samples.

# If Python lists are needed, use .tolist()
shap_std_sum_list = shap_std_sum_each_feature.tolist()
lime_std_sum_list = lime_std_sum_each_feature.tolist()

print("각 feature별 SHAP std 합 (길이 p):", shap_std_sum_list)
print("각 feature별 LIME std 합 (길이 p):", lime_std_sum_list)

각 feature별 SHAP std 합 (길이 p): [1.6353959762579373, 0.4652943055117794, 2.4635063203016876, 3.605101503605942, 1.2044506450431258, 0.1459028839431412, 0.19018452271894926, 0.1624623307187004, 0.11330356081312232, 0.10724681947183715]
각 feature별 LIME std 합 (길이 p): [0.06692954801752421, 0.03872961276675696, 0.6135925965690895, 0.8679351210320528, 0.05752777187285849, 0.09016409051313498, 0.08775053013205017, 0.0914356143663165, 0.08199209494140974, 0.08012904055744668]


In [ ]:
N_samples_for_test = 100        # Number of test samples (random selection)
n_runs = 40                     # Number of iterations per sample
p = X_train.shape[1]            # Number of features

# (Results) Arrays to accumulate the sum of standard deviations for each feature
shap_std_sum_each_feature = np.zeros(p)  
lime_std_sum_each_feature = np.zeros(p)  

# Randomly select 100 test samples (without replacement)
np.random.seed(41)  # Fix random seed for reproducibility
test_indices = np.random.choice(len(X_test), size=N_samples_for_test, replace=False)

for idx in test_indices:
    # Current test sample
    x_test_original = X_test.iloc[idx].values  # shape=(p,)

    # Lists to store SHAP and LIME results across runs
    shap_values_runs = []
    lime_coefs_runs = []

    for run_id in range(n_runs):
        # Add small noise to x_test_point
        x_test_perturbed = x_test_original + np.random.normal(0, 0.01, size=p)

        # Compute SHAP values
        _, shap_vals = kernel_shap_no_correction_single_mean_fill(
            model, 
            X_train.values, 
            x_test_perturbed
        )
        shap_values_runs.append(shap_vals)

        # Compute LIME coefficients (random_state fixed)
        coefs, _ = tabular_lime_explanation_regression(
            X_train.values, 
            model, 
            x_test_perturbed,
            num_samples=3000,
            random_state=42  
        )
        lime_coefs_runs.append(coefs)

    # Convert to NumPy arrays
    shap_values_runs = np.array(shap_values_runs)  # shape=(n_runs, p)
    lime_coefs_runs = np.array(lime_coefs_runs)    # shape=(n_runs, p)

    # Compute feature-wise standard deviation
    shap_std = shap_values_runs.std(axis=0)  # shape=(p,)
    lime_std = lime_coefs_runs.std(axis=0)   # shape=(p,)

    # Accumulate feature-wise standard deviations over 100 samples
    shap_std_sum_each_feature += shap_std
    lime_std_sum_each_feature += lime_std

# At the end, shap_std_sum_each_feature and lime_std_sum_each_feature (shape=(p,))
# contain the total sum of std values recorded for each feature across 100 samples.

# If Python lists are needed, use .tolist()
shap_std_sum_list = shap_std_sum_each_feature.tolist()
lime_std_sum_list = lime_std_sum_each_feature.tolist()

print("각 feature별 SHAP std 합 (길이 p):", shap_std_sum_list)
print("각 feature별 LIME std 합 (길이 p):", lime_std_sum_list)

각 feature별 SHAP std 합 (길이 p): [1.8443670359038722, 0.5263056628818887, 2.369896623498704, 4.078381324536716, 0.9988604851775671, 0.19575748841733667, 0.1820129666393052, 0.2665861800989916, 0.1690421875241291, 0.12105209826547984]
각 feature별 LIME std 합 (길이 p): [0.06471496939269901, 0.045084181017521145, 0.6228025981202929, 0.9518710939304331, 0.05652144688255208, 0.08569474051525917, 0.09064247218334816, 0.09947671189796813, 0.08221566371881076, 0.08119491512117116]
